## Import and merge normalized data from 5 annotators

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def load_annotated_file(no_annotator):
  df = pd.read_excel('/content/drive/MyDrive/Khai thác dữ liệu truyền thông xã hội - IE403.N22/Đồ án/Annotation/Annotator{}.xlsx'.format(no_annotator))
  df.columns = ['sent_idx', 'token_idx', 'token', 'normalized_{}'.format(no_annotator), 'is_normalized_{}'.format(no_annotator), 'uncertain_{}'.format(no_annotator)]
  df['sent_idx'] = df['token_idx']

  sent_idx = -1
  for row in range(len(df)):
    if('Index' in str(df.iloc[row, 0])):
      df.iloc[row, 0] = np.nan
      df.iloc[row, 1] = np.nan
      sent_idx += 1
    else:
      df.iloc[row, 0] = sent_idx

  df.dropna(how='all', inplace = True)
  df = df.set_index(['sent_idx', 'token_idx', 'token'])

  return df  

In [ ]:
annotator_1 = load_annotated_file(1)
annotator_2 = load_annotated_file(2)
annotator_3 = load_annotated_file(3)
annotator_4 = load_annotated_file(4)
annotator_5 = load_annotated_file(5)

print(annotator_1.shape)
print(annotator_2.shape)
print(annotator_3.shape)
print(annotator_4.shape)
print(annotator_5.shape)

(40174, 3)
(40174, 3)
(40174, 3)
(40174, 3)
(40174, 3)


In [ ]:
annotated_df = annotator_1.join(annotator_2).join(annotator_3).join(annotator_4).join(annotator_5)
annotated_df['norm1'] = np.nan
annotated_df['norm2'] = np.nan
annotated_df['label1'] = np.nan
annotated_df['label2'] = np.nan
annotated_df['uncertain1'] = np.nan
annotated_df['uncertain2'] = np.nan
annotated_df

normalized_1 is_normalized_1 uncertain_1  \
sent_idx token_idx token                                            
0        0         em              em             NaN         NaN   
         1         được          được             NaN         NaN   
         2         làm            làm             NaN         NaN   
         3         fan            fan             NaN         NaN   
         4         cứng          cứng             NaN         NaN   
...                               ...             ...         ...   
3399     30        có             NaN             NaN         NaN   
         31        lên            NaN             NaN         NaN   
         32        thiên          NaN             NaN         NaN   
         33        đường          NaN             NaN         NaN   
         34        nha            NaN             NaN         NaN   

                         normalized_2 is_normalized_2 uncertain_2  \
sent_idx token_idx token                                            
0        0         em              em             NaN         NaN   
         1         được          được             NaN         NaN   
         2         làm            làm             NaN         NaN   
         3         fan            fan             NaN         NaN   
         4         cứng          cứng             NaN         NaN   
...                               ...             ...         ...   
3399     30        có             NaN             NaN         NaN   
         31        lên            NaN             NaN         NaN   
         32        thiên          NaN             NaN         NaN   
         33        đường          NaN             NaN         NaN   
         34        nha            NaN             NaN         NaN   

                         normalized_3 is_normalized_3 uncertain_3  \
sent_idx token_idx token                                            
0        0         em             NaN             NaN         NaN   
         1         được           NaN             NaN         NaN   
         2         làm            NaN             NaN         NaN   
         3         fan            NaN             NaN         NaN   
         4         cứng           NaN             NaN         NaN   
...                               ...             ...         ...   
3399     30        có             NaN             NaN         NaN   
         31        lên            NaN             NaN         NaN   
         32        thiên          NaN             NaN         NaN   
         33        đường          NaN             NaN         NaN   
         34        nha            NaN             NaN         NaN   

                         normalized_4  ... uncertain_4 normalized_5  \
sent_idx token_idx token               ...                            
0        0         em             NaN  ...         NaN          NaN   
         1         được           NaN  ...         NaN          NaN   
         2         làm            NaN  ...         NaN          NaN   
         3         fan            NaN  ...         NaN          NaN   
         4         cứng           NaN  ...         NaN          NaN   
...                               ...  ...         ...          ...   
3399     30        có              có  ...         NaN           có   
         31        lên            lên  ...         NaN          lên   
         32        thiên        thiên  ...         NaN        thiên   
         33        đường        đường  ...         NaN        đường   
         34        nha            nha  ...         NaN          nha   

                         is_normalized_5 uncertain_5 norm1  norm2  label1  \
sent_idx token_idx token                                                    
0        0         em                NaN         NaN   NaN    NaN     NaN   
         1         được              NaN         NaN   NaN    NaN     NaN   
         2         làm               NaN         NaN   NaN    NaN     NaN   
         3         fan

In [ ]:
def encode_tag(sent_idx, annotator1, annotator2):
  df = annotated_df.loc[(annotated_df.index.get_level_values(0)%10 == sent_idx)].reset_index()

  for row in range(len(df)):
    labels = []
    values = []
    uncertain = []
    for i in [annotator1, annotator2]:
      if(df['is_normalized_{}'.format(i)][row] == '#'):
        labels.append(1)
        values.append(df['normalized_{}'.format(i)][row])
        if(df['uncertain_{}'.format(i)][row] == '?'):
          uncertain.append(1)
        else:
          uncertain.append(0)
      else:
        labels.append(0)
        values.append(df['normalized_{}'.format(i)][row])
        uncertain.append(0)
    df.loc[row, 'norm1'], df.loc[row, 'norm2'] = values[0], values[1]
    df.loc[row, 'label1'], df.loc[row, 'label2'] = labels[0], labels[1]
    df.loc[row, 'uncertain1'], df.loc[row, 'uncertain2'] = uncertain[0], uncertain[1]

  annotated_df.loc[(annotated_df.index.get_level_values(0)%10 == sent_idx), 'norm1'] = list(df['norm1'])
  annotated_df.loc[(annotated_df.index.get_level_values(0)%10 == sent_idx), 'norm2'] = list(df['norm2'])
  annotated_df.loc[(annotated_df.index.get_level_values(0)%10 == sent_idx), 'label1'] = list(df['label1'])
  annotated_df.loc[(annotated_df.index.get_level_values(0)%10 == sent_idx), 'label2'] = list(df['label2'])
  annotated_df.loc[(annotated_df.index.get_level_values(0)%10 == sent_idx), 'uncertain1'] = list(df['uncertain1'])
  annotated_df.loc[(annotated_df.index.get_level_values(0)%10 == sent_idx), 'uncertain2'] = list(df['uncertain2'])

In [ ]:
encode_tag(0, 1, 2)
encode_tag(1, 1, 3)
encode_tag(2, 1, 4)
encode_tag(3, 1, 5)
encode_tag(4, 2, 3)
encode_tag(5, 2, 4)
encode_tag(6, 2, 5)
encode_tag(7, 3, 4)
encode_tag(8, 3, 5)
encode_tag(9, 4, 5)

In [ ]:
annotated_df = annotated_df[['norm1', 'norm2', 'label1', 'label2', 'uncertain1', 'uncertain2']]
annotated_df

norm1  norm2  label1  label2  uncertain1  uncertain2
sent_idx token_idx token                                                      
0        0         em        em     em     0.0     0.0         0.0         0.0
         1         được    được   được     0.0     0.0         0.0         0.0
         2         làm      làm    làm     0.0     0.0         0.0         0.0
         3         fan      fan    fan     0.0     0.0         0.0         0.0
         4         cứng    cứng   cứng     0.0     0.0         0.0         0.0
...                         ...    ...     ...     ...         ...         ...
3399     30        có        có     có     0.0     0.0         0.0         0.0
         31        lên      lên    lên     0.0     0.0         0.0         0.0
         32        thiên  thiên  thiên     0.0     0.0         0.0         0.0
         33        đường  đường  đường     0.0     0.0         0.0         0.0
         34        nha      nha    nha     0.0     0.0         0.0         0.0

[40174 rows x 6 columns]

In [ ]:
# lowercase
annotated_df['norm1'] = annotated_df['norm1'].str.lower()
annotated_df['norm2'] = annotated_df['norm2'].str.lower()
# remove begin and end whitespace
annotated_df['norm1'] = annotated_df['norm1'].str.strip()
annotated_df['norm2'] = annotated_df['norm2'].str.strip()

<ipython-input-12-1a35cd85e797>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotated_df['norm1'] = annotated_df['norm1'].str.lower()
<ipython-input-12-1a35cd85e797>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotated_df['norm2'] = annotated_df['norm2'].str.lower()
<ipython-input-12-1a35cd85e797>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

## Calculate inner-annotator agreement

In [ ]:
from sklearn.metrics import cohen_kappa_score
cohen_kappa_score(annotated_df['label1'], annotated_df['label2'])

0.9013843357360888

In [ ]:
disagree_df = annotated_df.loc[(annotated_df['label1'] != 0) | (annotated_df['label2'] != 0)]
disagree_df

norm1  norm2  label1  label2  uncertain1  uncertain2
sent_idx token_idx token                                                      
1        12        t        tao    tao     1.0     1.0         0.0         0.0
         20        r        rồi    rồi     1.0     1.0         0.0         0.0
         23        t        tao    tao     1.0     1.0         0.0         0.0
         27        l        lồn     lờ     1.0     1.0         1.0         1.0
3        8         nghi    nghị   nghị     1.0     1.0         0.0         0.0
...                         ...    ...     ...     ...         ...         ...
3397     3         ko     không  không     1.0     1.0         0.0         0.0
3398     2         h        giờ    giờ     1.0     1.0         0.0         0.0
         4         j         gì    làm     1.0     1.0         0.0         0.0
         24        rẽ        rẻ     rẽ     1.0     0.0         0.0         0.0
3399     11        triêu  triệu  triệu     1.0     1.0         0.0         0.0

[4837 rows x 6 columns]

In [ ]:
def same_cand(df):
  count = 0
  for i in range(len(df)):
    if(df.reset_index()['norm1'][i] == df.reset_index()['norm2'][i]):
      count += 1
      #print(nsw_df.reset_index()['annotator1'][i], nsw_df.reset_index()['annotator2'][i])

  return(count/len(df))

In [ ]:
print(same_cand(disagree_df))

0.7448831920611949


## Export disagreed data file for annotator 6

In [ ]:
annotated_df['norm'] = np.nan
annotated_df = annotated_df.reset_index()

for i in range(len(annotated_df)):
  row = annotated_df.iloc[i,:]
  
  if(row['label1'] == row['label2']): # label1 = label2
    if(row['norm1'] == row['norm2']): # norm1 = norm2
      annotated_df['norm'][i] = row['norm1']
    else: # norm1 != norm2 (1 = 1)
      if(row['uncertain1'] != row['uncertain2']):
        if(row['uncertain1'] == 0):
          annotated_df['norm'][i] = row['norm1']
        else:
          annotated_df['norm'][i] = row['norm2']

annotated_df

<ipython-input-71-c7419486293a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotated_df['norm'] = np.nan
<ipython-input-71-c7419486293a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annotated_df['norm'][i] = row['norm1']


,sent_idx,token_idx,token,norm1,norm2,label1,label2,uncertain1,uncertain2,norm
0,0,0,em,em,em,0.0,0.0,0.0,0.0,em
1,0,1,được,được,được,0.0,0.0,0.0,0.0,được
2,0,2,làm,làm,làm,0.0,0.0,0.0,0.0,làm
3,0,3,fan,fan,fan,0.0,0.0,0.0,0.0,fan
4,0,4,cứng,cứng,cứng,0.0,0.0,0.0,0.0,cứng
...,...,...,...,...,...,...,...,...,...,...
40169,3399,30,có,có,có,0.0,0.0,0.0,0.0,có
40170,3399,31,lên,lên,lên,0.0,0.0,0.0,0.0,lên
40171,3399,32,thiên,thiên,thiên,0.0,0.0,0.0,0.0,thiên
40172,3399,33,đường,đường,đường,0.0,0.0,0.0,0.0,đường


In [ ]:
annotated_df.drop(['uncertain1', 'uncertain2'], axis = 1).to_excel("/content/drive/MyDrive/Khai thác dữ liệu truyền thông xã hội - IE403.N22/Đồ án/Annotation/annotator6.xlsx")

In [ ]:
# re-annotate